# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,agryz,56.5203,52.9942,7.64,86,26,3.79,RU,1712889345
1,1,la ronge,55.1001,-105.2842,2.14,69,20,3.09,CA,1712889345
2,2,hujra shah muqim,30.7418,73.8236,29.23,18,62,3.18,PK,1712889345
3,3,dalnegorsk,44.5604,135.5701,17.56,88,0,1.34,RU,1712889345
4,4,kununurra,-15.7667,128.7333,30.99,33,12,5.66,AU,1712889345


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
# %%capture --no-display

# Configure the map plot
humplot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.5, 
    color = "City"
)

# Display the map
humplot 

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
#prep to clean data
city_data_df.count()

City_ID       562
City          562
Lat           562
Lng           562
Max Temp      562
Humidity      562
Cloudiness    562
Wind Speed    562
Country       557
Date          562
dtype: int64

In [6]:
#cleaning data

city_data_clean = city_data_df.dropna(how="any")

In [7]:
#verifying data clean 

city_data_clean.count()

City_ID       557
City          557
Lat           557
Lng           557
Max Temp      557
Humidity      557
Cloudiness    557
Wind Speed    557
Country       557
Date          557
dtype: int64

In [8]:
# Narrow down cities that fit criteria and drop any results with null values
# Instructions allowed for individual preferences: Humidity <50, Cloudiness >30 and Max Temp <15 are my preferences

paradise = city_data_clean.loc[(city_data_clean['Humidity'] <50) & (city_data_clean['Cloudiness'] >30) & (city_data_clean['Max Temp'] <15)]

# Drop any rows with null values
# Done previously 

# Display sample data
paradise.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
16,16,moose jaw,50.4001,-105.5344,6.14,26,44,1.54,CA,1712889346
71,71,badger,64.8000,-147.5333,6.10,49,75,5.14,US,1712889246
74,74,fort mcmurray,56.7268,-111.3810,11.77,30,100,4.12,CA,1712889349
91,91,whitehorse,60.7161,-135.0538,6.43,28,75,7.72,CA,1712889350
97,97,khilok,51.3674,110.4676,9.48,43,100,2.65,RU,1712889350


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_full= paradise.copy()
hotels_df = hotel_full.drop(columns=['City_ID', 'Max Temp', 'Cloudiness', 'Wind Speed', 'Date'])
headers = ["City", "Country", "Lat", "Lng", "Humidity", "Hotel Name"]
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df = hotels_df.reindex(columns= headers)

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
16,moose jaw,CA,50.4001,-105.5344,26,NaN
71,badger,US,64.8000,-147.5333,49,NaN
74,fort mcmurray,CA,56.7268,-111.3810,30,NaN
91,whitehorse,CA,60.7161,-135.0538,28,NaN
97,khilok,RU,51.3674,110.4676,43,NaN


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
limit = 20

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    filters = f"circle:{longitude},{latitude},{radius}"
    bias = f"proximity:{longitude},{latitude}"

    params = {
    "categories":categories,
    "limit":limit,
    "filter":filters,
    "bias":bias,
    "apiKey":geoapify_key
}
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
moose jaw - nearest hotel: Temple Gardens Mineral Spa Resort
badger - nearest hotel: Clarion Hotel & Suites
fort mcmurray - nearest hotel: Nomad Hotel and Suites
whitehorse - nearest hotel: Town & Mountain Hotel
khilok - nearest hotel: Viola
happy valley-goose bay - nearest hotel: Hotel North Two
shihezi - nearest hotel: 石河子宾馆
fort st. john - nearest hotel: Fort St. John Motor Inn
general conesa - nearest hotel: Gran Hotel Conesa
college - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
ozieri - nearest hotel: Hotel Mastino
levuka - nearest hotel: No hotel found
undurkhaan - nearest hotel: Khanburged Hotel
tosontsengel - nearest hotel: Алтай
dzhalagash - nearest hotel: No hotel found
dalandzadgad - nearest hotel: Хан Уул буудал
semnan - nearest hotel: مرکز آموزشی رفاهی فرهنگیان - خانه معلم
erdenet - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
16,moose jaw,CA,50.4001,-105.5344,26,Temple Gardens Mineral Spa Resort
71,badger,US,64.8000,-147.5333,49,Clarion Hotel & Suites
74,fort mcmurray,CA,56.7268,-111.3810,30,Nomad Hotel and Suites
91,whitehorse,CA,60.7161,-135.0538,28,Town & Mountain Hotel
97,khilok,RU,51.3674,110.4676,43,Viola
111,happy valley-goose bay,CA,53.3168,-60.3315,46,Hotel North Two
114,shihezi,CN,44.3000,86.0333,49,石河子宾馆
146,fort st. john,CA,56.2499,-120.8529,43,Fort St. John Motor Inn
165,general conesa,AR,-40.1000,-64.4333,44,Gran Hotel Conesa
249,college,US,64.8569,-147.8028,49,BEST WESTERN PLUS Pioneer Park Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [14]:
#%%capture --no-display

# Configure the map plot
hotplot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.5,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
    
)

# Display the map
hotplot 

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)